In [1]:
import os
if os.getcwd() == '/home/user/code':
    os.chdir('/home/user/code/nlp2024_ClefTask4SOTA')

In [2]:
run_id = "baseline-tdms-valid_llama3_70b-05202024-224045"

path = f"results/{run_id}/df.feather"

import pandas as pd


df = pd.read_feather(path)


In [3]:
from src.dataset import TDMSDataset, PATH, UNANSWERABLE, LogResult
valid_dataset = TDMSDataset(PATH.VAL)
df_ground_truth = pd.DataFrame([{"f":f, "ground_truth":gt} for f, _, gt in [valid_dataset.__getitem__(i) for i in range(len(df))]])
df = df_ground_truth.merge(df, on="f")

In [4]:
from scoring_program.evaluation import Metrics



results = Metrics.evaluate_property_wise_json_based(
    label_list=list(df["ground_truth"]), prediction_list=list(df["annotation"])
)

results.update(
    Metrics.evaluate_rouge(label_list=list(df["ground_truth"]), prediction_list=list(df["annotation"]))
)

2024-05-21 08:37:35.219016: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/user/.venv/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [10]:
results

{'general_accuracy': 80.0,
 'exact_recalls_task': 30.43,
 'exact_recalls_dataset': 65.22,
 'exact_recalls_metric': 30.43,
 'exact_recalls_Score': 0.0,
 'exact_recalls_overall': 31.52,
 'partial_recalls_task': 91.3,
 'partial_recalls_dataset': 65.22,
 'partial_recalls_metric': 47.83,
 'partial_recalls_Score': 4.35,
 'partial_recalls_overall': 52.17,
 'exact_precisions_task': 3.04,
 'exact_precisions_dataset': 7.25,
 'exact_precisions_metric': 3.45,
 'exact_precisions_Score': 0.0,
 'exact_precisions_overall': 3.43,
 'partial_precisions_task': 9.13,
 'partial_precisions_dataset': 7.25,
 'partial_precisions_metric': 5.42,
 'partial_precisions_Score': 0.52,
 'partial_precisions_overall': 5.58,
 'exact_f1s_task': 5.53,
 'exact_f1s_dataset': 13.04,
 'exact_f1s_metric': 6.19,
 'exact_f1s_Score': 0.0,
 'exact_f1s_overall': 6.19,
 'partial_f1s_task': 16.6,
 'partial_f1s_dataset': 13.04,
 'partial_f1s_metric': 9.73,
 'partial_f1s_Score': 0.93,
 'partial_f1s_overall': 10.08,
 'rouge1': 20.28,
 'ro

In [15]:
pd.DataFrame([results]).T

,0
general_accuracy,80.00
exact_recalls_task,30.43
exact_recalls_dataset,65.22
exact_recalls_metric,30.43
exact_recalls_Score,0.00
exact_recalls_overall,31.52
partial_recalls_task,91.30
partial_recalls_dataset,65.22
partial_recalls_metric,47.83
partial_recalls_Score,4.35


In [5]:
# !pip uninstall evaluate -y


Found existing installation: evaluate 0.4.2
Uninstalling evaluate-0.4.2:
  Successfully uninstalled evaluate-0.4.2


In [13]:
# !pip install openpyxl
# 


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 5.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
